In [207]:
import pandas as pd
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [208]:
canada = pd.read_csv('CAvideos.csv')
germany = pd.read_csv('DEvideos.csv')
france = pd.read_csv('FRvideos.csv')
uk = pd.read_csv('GBvideos.csv')
india = pd.read_csv('INvideos.csv')
japan = pd.read_csv('JPvideos.csv', engine='python')
korea = pd.read_csv('KRvideos.csv', engine='python')
mexico = pd.read_csv('MXvideos.csv', engine='python')
russia = pd.read_csv('RUvideos.csv', engine='python')
us = pd.read_csv('USvideos.csv')

In [209]:
#Function that reads json file and returns dictionary with category id as key and category title as value
def read_json_files(filename):
    json = pd.read_json(filename)
    Dict = {}
    for item in json['items']: 

        category_id = item['id']
        category = item['snippet']['title']
        Dict[category_id] = category
        
    #check if category id 29 exists in dict - for some regions this id is missing in the json file for some reason
    if not('29' in Dict):
        Dict['29'] = 'Nonprofits & Activism'
        
    return Dict


In [227]:
def clean_data(df, json_file):

    df_clean = df.copy()
    df_clean = df_clean.drop(['thumbnail_link'], axis=1)

    #change the type of date columns from object to datetime64[ns] to be able to change the formatting
    #publish time - timezone is UTC - coordinated universal time
    df_clean['trending_date'] = pd.to_datetime(df_clean['trending_date'], format = '%y.%d.%m')
    df_clean['publish_time'] = pd.to_datetime(df_clean['publish_time'], format = '%Y-%m-%dT%H:%M:%S.%f%z')

    #change the formatting of date columns
    df_clean['trending_date'] = df_clean['trending_date'].dt.strftime('%Y-%m-%d').astype('datetime64[ns]')
    df_clean['publish_time'] = df_clean['publish_time'].dt.strftime('%Y-%m-%d %H:%M:%S').astype('datetime64[ns]')

    #cleaning up tags column
    #replaces |" (first tag is not enclosed with "") and "|" with ',' and removes last trailing character "
    df_clean['tags'] = df_clean['tags'].replace(to_replace='"\|"|\|"',value=',', regex=True).str.rstrip('"')
    
    #replace the value [none] in tags column with NaN
    df_clean['tags'].replace('[none]', np.nan, inplace=True)
    
    #replace value 'Deleted video' in title column with NaN
    df_clean['title'].replace('Deleted video', np.nan, inplace=True)
    
    #create dictionary of category id's and values
    category_dict = read_json_files(json_file)

    #change category_id type to string and rename column
    df_clean['category_id'] = df_clean['category_id'].astype(str)
    df_clean = df_clean.rename(columns={'category_id':'category'})

    #replace id's with category name
    df_clean['category'] = df_clean['category'].apply(category_dict.get)
    
    return df_clean



In [228]:
canada_clean = clean_data(canada, 'CA_category_id.json')
germany_clean = clean_data(germany, 'DE_category_id.json')
france_clean = clean_data(france, 'FR_category_id.json')
uk_clean = clean_data(uk, 'GB_category_id.json')
india_clean = clean_data(india, 'IN_category_id.json')
japan_clean = clean_data(japan, 'JP_category_id.json')
korea_clean = clean_data(korea, 'KR_category_id.json')
mexico_clean = clean_data(mexico, 'MX_category_id.json')
russia_clean = clean_data(russia, 'RU_category_id.json')
us_clean = clean_data(us, 'US_category_id.json')


In [229]:
canada_clean.head()
germany_clean.head()
france_clean.head()
uk_clean.head()
india_clean.head()
japan_clean.head()
korea_clean.head()
mexico_clean.head()
russia_clean.head()
us_clean.head()

,video_id,trending_date,title,channel_title,category,publish_time,tags,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled,video_error_or_removed,description
0,n1WpP7iowLc,2017-11-14,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,Music,2017-11-10 17:00:03,"Eminem,Walk,On,Water,Aftermath/Shady/Interscop...",17158579,787425,43420,125882,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...
1,0dBIkQ4Mz1M,2017-11-14,PLUSH - Bad Unboxing Fan Mail,iDubbbzTV,Comedy,2017-11-13 17:00:00,"plush,bad unboxing,unboxing,fan mail,idubbbztv...",1014651,127794,1688,13030,False,False,False,STill got a lot of packages. Probably will las...
2,5qpjK5DgCt4,2017-11-14,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,Comedy,2017-11-12 19:05:24,"racist superman,rudy,mancuso,king,bach,racist,...",3191434,146035,5339,8181,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,d380meD0W0M,2017-11-14,I Dare You: GOING BALD!?,nigahiga,Entertainment,2017-11-12 18:01:41,"ryan,higa,higatv,nigahiga,i dare you,idy,rhpc,...",2095828,132239,1989,17518,False,False,False,I know it's been a while since we did this sho...
4,2Vv-BfVoq4g,2017-11-14,Ed Sheeran - Perfect (Official Music Video),Ed Sheeran,Music,2017-11-09 11:04:14,"edsheeran,ed sheeran,acoustic,live,cover,offic...",33523622,1634130,21082,85067,False,False,False,🎧: https://ad.gt/yt-perfect\n💰: https://atlant...


,video_id,trending_date,title,channel_title,category,publish_time,tags,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled,video_error_or_removed,description
0,LgVi6y5QIjM,2017-11-14,Sing zu Ende! | Gesangseinlagen vom Feinsten |...,inscope21,Entertainment,2017-11-13 17:08:49,"inscope21,sing zu ende,gesangseinlagen,gesang,...",252786,35885,230,1539,False,False,False,Heute gibt es mal wieder ein neues Format... w...
1,Bayt7uQith4,2017-11-14,Kinder ferngesteuert im Kiosk! Erwachsene abzo...,LUKE! Die Woche und ich,Comedy,2017-11-12 22:30:01,"Kinder,ferngesteuert,Kinder ferngesteuert,luke...",797196,53576,302,1278,False,False,False,Kinder ferngesteuert! Kinder lassen sich sooo ...
2,1ZAPwfrtAFY,2017-11-14,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,Entertainment,2017-11-13 07:30:00,"last week tonight trump presidency,last week t...",2418783,97190,6146,12703,False,False,False,"One year after the presidential election, John..."
3,AHtypnRk7JE,2017-11-14,Das Fermi-Paradoxon,100SekundenPhysik,Education,2017-11-12 15:00:01,"Physik,Wissenschaft,Technik,Science-Fiction,Au...",380247,31821,458,1955,False,False,False,►Alle Videos: http://bit.ly/1fa7Tw3\n\n\n✚Snap...
4,ZJ9We4bjcg0,2017-11-14,18 SONGS mit Kelly MissesVlog (Sing-off),rezo,Entertainment,2017-11-12 13:10:36,"kelly,missesvlog,kelly song,bausa,bausa was du...",822213,100684,2467,10244,False,False,False,18 Song Mashup über den (veränderten) Beat von...


,video_id,trending_date,title,channel_title,category,publish_time,tags,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled,video_error_or_removed,description
0,Ro6eob0LrCY,2017-11-14,Malika LePen : Femme de Gauche - Trailer,Le Raptor Dissident,Entertainment,2017-11-13 17:32:55,"Raptor,Dissident,Expliquez,moi,cette,merde",212702,29282,1108,3817,False,False,False,Dimanche.\n18h30.\nSoyez présents pour la vidé...
1,Yo84eqYwP98,2017-11-14,"LA PIRE PARTIE ft Le Rire Jaune, Pierre Croce,...",Le Labo,Entertainment,2017-11-12 15:00:02,NaN,432721,14053,576,1161,False,False,False,Le jeu de société: https://goo.gl/hhG1Ta\n\nGa...
2,ceqntSXE-10,2017-11-14,DESSINS ANIMÉS FRANÇAIS VS RUSSES 2 - Daniil...,Daniil le Russe,Comedy,2017-11-13 17:00:38,"cartoon,pokémon,école,ours,мультфильм",482153,76203,477,9580,False,False,False,Une nouvelle dose de dessins animés français e...
3,WuTFI5qftCE,2017-11-14,PAPY GRENIER - METAL GEAR SOLID,Joueur Du Grenier,Gaming,2017-11-12 17:00:02,"Papy grenier,Metal Gear Solid,PS1,Tirage d'ore...",925222,85016,550,4303,False,False,False,"Nouvel ,épisode de Papy Grenier ! Ce mois-ci o..."
4,ee6OFs8TdEg,2017-11-14,QUI SAUTERA LE PLUS HAUT ? (VÉLO SKATE ROLLER ...,Aurelien Fontenoy,Sports,2017-11-13 16:30:03,"vélo,vtt,bmx,freestyle,bike,mtb,dirt,trottinet...",141695,8091,72,481,False,False,False,Sauts à plus de 4 mètres de haut dans un tramp...


,video_id,trending_date,title,channel_title,category,publish_time,tags,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled,video_error_or_removed,description
0,Jw1Y-zhQURU,2017-11-14,John Lewis Christmas Ad 2017 - #MozTheMonster,John Lewis,Howto & Style,2017-11-10 07:38:29,"christmas,john lewis christmas,john lewis,chri...",7224515,55681,10247,9479,False,False,False,Click here to continue the story and make your...
1,3s1rvMFUweQ,2017-11-14,Taylor Swift: …Ready for It? (Live) - SNL,Saturday Night Live,Entertainment,2017-11-12 06:24:44,"SNL,Saturday Night Live,SNL Season 43,Episode ...",1053632,25561,2294,2757,False,False,False,Musical guest Taylor Swift performs …Ready for...
2,n1WpP7iowLc,2017-11-14,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,Music,2017-11-10 17:00:03,"Eminem,Walk,On,Water,Aftermath/Shady/Interscop...",17158579,787420,43420,125882,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...
3,PUTEiSjKwJU,2017-11-14,Goals from Salford City vs Class of 92 and Fri...,Salford City Football Club,Sports,2017-11-13 02:30:38,"Salford City FC,Salford City,Salford,Class of ...",27833,193,12,37,False,False,False,Salford drew 4-4 against the Class of 92 and F...
4,rHwDegptbI4,2017-11-14,Dashcam captures truck's near miss with child ...,Cute Girl Videos,News & Politics,2017-11-13 01:45:13,NaN,9815,30,2,30,False,False,False,Dashcam captures truck's near miss with child ...


,video_id,trending_date,title,channel_title,category,publish_time,tags,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled,video_error_or_removed,description
0,kzwfHumJyYc,2017-11-14,Sharry Mann: Cute Munda ( Song Teaser) | Parmi...,Lokdhun Punjabi,Film & Animation,2017-11-12 12:20:39,"sharry mann,sharry mann new song,sharry mann c...",1096327,33966,798,882,False,False,False,Presenting Sharry Mann latest Punjabi Song Cu...
1,zUZ1z7FwLc8,2017-11-14,"पीरियड्स के समय, पेट पर पति करता ऐसा, देखकर दं...",HJ NEWS,News & Politics,2017-11-13 05:43:56,"पीरियड्स के समय,पेट पर पति करता ऐसा,देखकर दंग ...",590101,735,904,0,True,False,False,"पीरियड्स के समय, पेट पर पति करता ऐसा, देखकर दं..."
2,10L1hZ9qa58,2017-11-14,Stylish Star Allu Arjun @ ChaySam Wedding Rece...,TFPC,Entertainment,2017-11-12 15:48:08,Stylish Star Allu Arjun @ ChaySam Wedding Rece...,473988,2011,243,149,False,False,False,Watch Stylish Star Allu Arjun @ ChaySam Weddin...
3,N1vE8iiEg64,2017-11-14,Eruma Saani | Tamil vs English,Eruma Saani,Comedy,2017-11-12 07:08:48,"Eruma Saani,Tamil Comedy Videos,Films,Movies,H...",1242680,70353,1624,2684,False,False,False,This video showcases the difference between pe...
4,kJzGH0PVQHQ,2017-11-14,why Samantha became EMOTIONAL @ Samantha naga ...,Filmylooks,Entertainment,2017-11-13 01:14:16,"Filmylooks,latest news,telugu movies,telugu ne...",464015,492,293,66,False,False,False,why Samantha became EMOTIONAL @ Samantha naga ...


,video_id,trending_date,title,channel_title,category,publish_time,tags,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled,video_error_or_removed,description
0,5ugKfHgsmYw,2018-02-07,é™¸è‡ªãƒ˜ãƒªã€�åž‚ç›´ã�«è�½ä¸‹ï¼�è·¯ä¸Šã�®è»Šã...,æ™‚äº‹é€šä¿¡æ˜ åƒ�ã‚»ãƒ³ã‚¿ãƒ¼,News & Politics,2018-02-06 03:04:37,"äº‹æ•…,ä½�è³€,ä½�è³€çœŒ,ãƒ˜ãƒªã‚³ãƒ—ã‚¿ãƒ¼,ãƒ˜...",188085,591,189,0,True,False,False,ä½�è³€çœŒç¥žåŸ¼å¸‚ã�®æ°‘å®¶ã�«å¢œè�½ã�—ã�Ÿé™¸ä...
1,ohObafdd34Y,2018-02-07,ã‚¤ãƒƒãƒ†Q ã�Šç¥­ã‚Šç”·å®®å·�Ã—æ‰‹è¶Š å·¨å¤§ãƒ...,ç¥žè°·ã�ˆã‚Šã�ª Kamiya Erina 2,Film & Animation,2018-02-06 04:01:56,NaN,90929,442,88,174,False,False,False,NaN
2,aBr2kKAHN6M,2018-02-07,Live Views of Starman,SpaceX,Science & Technology,2018-02-06 21:38:22,NaN,6408303,165892,2331,3006,False,False,False,NaN
3,5wNnwChvmsQ,2018-02-07,æ�±äº¬ãƒ‡ã‚£ã‚ºãƒ‹ãƒ¼ãƒªã‚¾ãƒ¼ãƒˆã�®å…ƒã‚­ãƒ£ã...,ã‚¢ã‚·ã‚¿ãƒŽãƒ¯ãƒ€ã‚¤,News & Politics,2018-02-06 06:08:49,ã‚¢ã‚·ã‚¿ãƒŽãƒ¯ãƒ€ã‚¤,96255,1165,277,545,False,False,False,æ�±äº¬ãƒ‡ã‚£ã‚ºãƒ‹ãƒ¼ãƒªã‚¾ãƒ¼ãƒˆã�®å…ƒã‚­ãƒ£ã...
4,B7J47qFvdsk,2018-02-07,æ¦®å€‰å¥ˆã€…ã€�è¡�æ’ƒã�®æ­»ã‚“ã� ã�µã‚Šï¼�æ˜ ç...,ã‚·ãƒ�ãƒžãƒˆã‚¥ãƒ‡ã‚¤,Film & Animation,2018-02-06 02:30:00,NaN,108408,1336,74,201,False,False,False,å®¶ã�«å¸°ã�£ã�¦ã��ã�Ÿã‚µãƒ©ãƒªãƒ¼ãƒžãƒ³ã�®ã�˜ã...


,video_id,trending_date,title,channel_title,category,publish_time,tags,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled,video_error_or_removed,description
0,RxGQe4EeEpA,2017-11-14,ì¢‹ì•„ by ë¯¼ì„œ_ìœ¤ì¢…ì‹ _ì¢‹ë‹ˆ ë‹µê°€,ë�¼í‘¸ë§ˆì½”ë¦¬ì•„,People & Blogs,2017-11-13 07:07:36,"ë�¼í‘¸ë§ˆ,ìœ¤ì¢…ì‹ ,ì¢‹ë‹ˆ,ì¢‹ì•„,ìƒ¬ë ˆ,ë¯¼ì„œ",156130,1422,40,272,False,False,False,ìœ¤ì¢…ì‹ 'ì¢‹ë‹ˆ'ì�˜ ë‹µê°€ 'ì¢‹ì•„' ìµœì´ˆ ê...
1,hH7wVE8OlQ0,2017-11-14,JSA ê·€ìˆœ ë¶�í•œêµ° ì´�ê²© ë¶€ìƒ�,Edward,News & Politics,2017-11-13 10:59:16,"JSA,ê·€ìˆœ,ë¶�í•œêµ°,ì´�ê²©,ë¶€ìƒ�,JSA ê·€ìˆœ ...",76533,211,28,113,False,False,False,[ì±„ë„�Aë‹¨ë�…]åŒ— ë³‘ì‚¬ í˜„ìž¬ 'ì�˜ì‹�ë¶ˆëª…...
2,9V8bnWUmE9U,2017-11-14,ë‚˜ëª°ë�¼íŒ¨ë°€ë¦¬ ìš´ë�™í™” ì˜�ìƒ� 2íƒ„ (ë¹¼ë...,ë‚˜ëª°ë�¼íŒ¨ë°€ë¦¬ í•«ì‡¼,People & Blogs,2017-11-11 07:16:08,"ì•„ë””ë‹¤ìŠ¤,ë¹¼ë¹¼ë¡œ,í•«ì‡¼,ë‚˜ëª°ë�¼íŒ¨ë°€ë...",421409,5112,166,459,False,False,False,í�¼ê°€ì‹¤ë•Œ ê¼­ ì¶œì²˜ ë¶€íƒ�ë“œë ¤ìš”
3,0_8py-t5R80,2017-11-14,"á„‹á…µá„†á…§á†¼á„‡á…¡á†¨ ì¶œêµ­ í˜„ìž¥, ë†“ì¹˜...",ë¯¸ë””ì–´ëª½êµ¬,News & Politics,2017-11-12 11:19:52,"ì�´ëª…ë°•,ì�´ëª…ë°• ì¶œêµ­ê¸ˆì§€,ì�´ëª…ë°• ì¶œ...",222850,2093,173,1219,False,False,False,ë‹¤ìŠ¤ëŠ” ëˆ„êµ¬ê²�ë‹ˆê¹Œ ë“£ê³ ë‚œ í›„ í‘œì •
4,bk55RbxiQdI,2017-11-14,ê¹€ìž¥ê²¸ì�€ ë¬¼ëŸ¬ê°”ë‹¤ MBC ë…¸ì¡° í™˜í˜¸ì™€...,NocutV,News & Politics,2017-11-13 11:08:59,"nocutV,ë…¸ì»·V,CBS,mbc,ê¹€ìž¥ê²¸,í•´ìž„,í—ˆì�¼í›„",84466,1094,109,450,False,False,False,ê¹€ìž¥ê²¸ MBC ì‚¬ìž¥ì�´ ê²°êµ­ í•´ìž„ë��ë‹¤.\n...


,video_id,trending_date,title,channel_title,category,publish_time,tags,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled,video_error_or_removed,description
0,SbOwzAl9ZfQ,2017-11-14,CapÃ­tulo 12 | MasterChef 2017,MasterChef 2017,Entertainment,2017-11-13 06:06:22,"MasterChef Junior 2017,TV Azteca,recetas,cocin...",310130,4182,361,1836,False,False,False,Disfruta la presencia del Chef Torreblanca en ...
1,klOV6Xh-DnI,2017-11-14,ALEXA EX-INTEGRANTE DEL GRUPO TIMBIRICHE RENUN...,Micky Contreras Martinez,People & Blogs,2017-11-13 05:11:58,La Voz Mexico 7,104972,271,174,369,False,False,False,ALEXA EX-INTEGRANTE DEL GRUPO TIMBIRICHE RENUN...
2,6L2ZF7Qzsbk,2017-11-14,LOUIS CKAGÃ“ - EL PULSO DE LA REPÃšBLICA,El Pulso De La RepÃºblica,News & Politics,2017-11-13 17:00:02,"Chumel Torres,El Pulso de la Republica,noticia...",136064,10105,266,607,False,False,False,La canciÃ³n del principio se llama â€œEste esp...
3,hcY52MFWMDM,2017-11-14,Sismo de 6.7 sacude Costa Rica 12 Noviembre 2017,Casanare,News & Politics,2017-11-13 03:47:10,"temblor,costa rica,sismo en costa rica",96153,378,171,208,False,False,False,El video es de un Walmart en el pais centroame...
4,_OXDcGPVAa4,2017-11-14,DOG HACKS | MUSAS LESSLIE LOS POLINESIOS,Musas,Howto & Style,2017-11-13 19:17:48,"MUSAS,lesslie,karen,hacks,perros,dogs,trucos,l...",499965,57781,681,7428,False,False,False,MI HERMANO NARRA MI RUTINA DE MAQUILLAJE\nhttp...


,video_id,trending_date,title,channel_title,category,publish_time,tags,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled,video_error_or_removed,description
0,gDuslQ9avLc,2017-11-14,Ð—Ð°Ñ…Ð°Ñ€ Ð¸ ÐŸÐ¾Ð»Ð¸Ð½Ð° ÑƒÑ‡Ð°Ñ‚Ñ�Ñ� Ñ�ÐºÐ¾...,Ð¢â€”Ð– Ð‘ÐžÐ“Ð�Ð§,People & Blogs,2017-11-13 09:09:31,"Ð·Ð°Ñ…Ð°Ñ€ Ð¸ Ð¿Ð¾Ð»Ð¸Ð½Ð°,ÑƒÑ‡Ð¸Ð¼Ñ�Ñ� Ñ�ÐºÐ¾...",62408,334,190,50,False,False,False,"Ð—Ð½Ð°ÐºÐ¾Ð¼ÑŒÑ‚ÐµÑ�ÑŒ, Ñ�Ñ‚Ð¾ Ð—Ð°Ñ…Ð°Ñ€ Ð¸ Ð..."
1,AOCJIFEA_jE,2017-11-14,Ð‘Ð¸Ñ€Ð¶Ð° ÐœÐµÐ¼Ð¾Ð² #29. Ð‘Ð¾Ð»ÑŒÑˆÐ¾Ð¸Ì† Ð²...,Druzhko Show,People & Blogs,2017-11-13 17:32:11,"Ð±Ð¸Ñ€Ð¶Ð° Ð¼ÐµÐ¼Ð¾Ð²,Ð»ÐµÐ² ÑˆÐ°Ð³Ð¸Ð½Ñ�Ð½,Ð¼...",330043,43841,2244,2977,False,False,False,Ð’ 29 Ð²Ñ‹Ð¿ÑƒÑ�ÐºÐµ Ð”Ñ€ÑƒÐ¶ÐºÐ¾ Ð¨Ð¾Ñƒ Ð¡ÐµÑ...
2,VAWNQDgwwOM,2017-11-14,Ð¥Ð�Ð™ÐŸ ÐšÐ­ÐœÐŸ - Ð¡Ð’ÐžÐ™ Ð¡Ð Ð•Ð”Ð˜ Ð§Ð£Ð–...,Ð®Ð»Ð¸Ðº,Entertainment,2017-11-13 16:11:31,"ÑŽÐ¼Ð¾Ñ€,ÐºÐ¾Ð¼ÐµÐ´Ð¸Ñ�,Ð²Ð»Ð¾Ð³,Ð±Ð»Ð¾Ð³,Ñ�Ð¼...",424596,49854,714,2944,False,False,False,http://kapitany.ru/!Yulik.cap - Ð¤Ð°ÐºÑƒÐ»ÑŒÑ‚...
3,gknkFwKQfHg,2017-11-14,Ð¡Ð¾Ñ‡Ð½Ð°Ñ� ÐºÐµÑ�Ð°Ð´Ð¸Ð»ÑŒÑ� Ñ� ÐºÑƒÑ€Ð¸Ñ†ÐµÐ¹,Hochland,People & Blogs,2017-11-13 06:51:10,"Ñ…Ð¾Ñ…Ð»Ð°Ð½Ð´,Ñ�Ñ‹Ñ€,Ñ€ÐµÑ†ÐµÐ¿Ñ‚Ñ‹,ÐºÐ°Ðº Ð¿...",112851,3566,122,80,False,False,False,*** ÐºÐ°Ðº Ð³Ð¾Ñ‚Ð¾Ð²Ð¸Ñ‚ÑŒ Ñ€ÐµÑ�Ñ‚Ð¾Ñ€Ð°Ð½Ð½...
4,3sYvZcwzZr8,2017-11-14,ÐšÐ›Ð˜ÐŸÐ« Ð ÐžÐ”Ð˜Ð¢Ð•Ð›Ð•Ð™ Ð�Ð� Ð¨ÐšÐžÐ›Ð¬Ð...,Ð¡Ð¾Ð²ÐµÑ€Ð³Ð¾Ð½,Entertainment,2017-11-13 16:52:36,"Ð¡Ð¾Ð²ÐµÑ€Ð³Ð¾Ð½,Sovergon,ÐºÐ»Ð¸Ð¿Ñ‹,Ñ€Ð¾Ð´Ð¸Ñ...",243469,36216,631,1692,False,False,False,â�� ÐšÐžÐ�ÐšÐ£Ð Ð¡ ÐšÐ�ÐŸÐ˜Ð¢Ð�Ð�Ð«: http://ka...


,video_id,trending_date,title,channel_title,category,publish_time,tags,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,2017-11-14,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,People & Blogs,2017-11-13 17:13:01,SHANtell martin,748374,57527,2966,15954,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,2017-11-14,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,Entertainment,2017-11-13 07:30:00,"last week tonight trump presidency,last week t...",2418783,97185,6146,12703,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,2017-11-14,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,Comedy,2017-11-12 19:05:24,"racist superman,rudy,mancuso,king,bach,racist,...",3191434,146033,5339,8181,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,2017-11-14,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,Entertainment,2017-11-13 11:00:04,"rhett and link,gmm,good mythical morning,rhett...",343168,10172,666,2146,False,False,False,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,2017-11-14,I Dare You: GOING BALD!?,nigahiga,Entertainment,2017-11-12 18:01:41,"ryan,higa,higatv,nigahiga,i dare you,idy,rhpc,...",2095731,132235,1989,17518,False,False,False,I know it's been a while since we did this sho...


In [233]:
ca = canada_clean.copy()
ca['country'] = 'Canada'

de = germany_clean.copy()
de['country'] = 'Germany'

fr = france_clean.copy()
fr['country'] = 'France'

gb = uk_clean.copy()
gb['country'] = 'Great Britain'

ind = india_clean.copy()
ind['country'] = 'India'

jp = japan_clean.copy()
jp['country'] = 'Japan'

kr = korea_clean.copy()
kr['country'] = 'Korea'

mx = mexico_clean.copy()
mx['country'] = 'Mexico'

ru = russia_clean.copy()
ru['country'] = 'Russia'

usa = us_clean.copy()
usa['country'] = 'USA'

all_data = pd.concat([ca, de, fr, gb, ind, jp, kr, mx, ru, usa])

all_data.head()

all_data.shape

,video_id,trending_date,title,channel_title,category,publish_time,tags,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled,video_error_or_removed,description,country
0,n1WpP7iowLc,2017-11-14,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,Music,2017-11-10 17:00:03,"Eminem,Walk,On,Water,Aftermath/Shady/Interscop...",17158579,787425,43420,125882,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...,Canada
1,0dBIkQ4Mz1M,2017-11-14,PLUSH - Bad Unboxing Fan Mail,iDubbbzTV,Comedy,2017-11-13 17:00:00,"plush,bad unboxing,unboxing,fan mail,idubbbztv...",1014651,127794,1688,13030,False,False,False,STill got a lot of packages. Probably will las...,Canada
2,5qpjK5DgCt4,2017-11-14,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,Comedy,2017-11-12 19:05:24,"racist superman,rudy,mancuso,king,bach,racist,...",3191434,146035,5339,8181,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...,Canada
3,d380meD0W0M,2017-11-14,I Dare You: GOING BALD!?,nigahiga,Entertainment,2017-11-12 18:01:41,"ryan,higa,higatv,nigahiga,i dare you,idy,rhpc,...",2095828,132239,1989,17518,False,False,False,I know it's been a while since we did this sho...,Canada
4,2Vv-BfVoq4g,2017-11-14,Ed Sheeran - Perfect (Official Music Video),Ed Sheeran,Music,2017-11-09 11:04:14,"edsheeran,ed sheeran,acoustic,live,cover,offic...",33523622,1634130,21082,85067,False,False,False,🎧: https://ad.gt/yt-perfect\n💰: https://atlant...,Canada


(375942, 16)